# Equity data collection
To build the ML prediction model, we need to collect data that represents each company. Based on research by Nikola Milosevic, the best choice of indicators are: 
-book value 
-market capitalisation 
-dividend yield 
-earnings per share 
-price to equity ratio 
-price to book ratio -debt to price ratio 
-current ratio 
-quick ratio 
-total debt to total equity ratio 
-current price 
-share price growth over the last year

Using these indicators, companies are seperating into two groups. These groups are companies that have share price growth of atleast 10%, and companies that don't.

In [1]:
# Import  all required library packages
import matplotlib.pyplot as plt, pandas as pd
import simfin as sf, numpy as np, datetime, math
from simfin.names import *
from datetime import date

In [2]:
# Set a data directory to download and store data
sf.set_data_dir('~/simfin_data_all/')
# Use the free api key
sf.set_api_key(api_key='free')

In [3]:
#Load the income statement
income_statement = sf.load(dataset='income',variant='annual', market='us',index=[TICKER], parse_dates=[REPORT_DATE, PUBLISH_DATE, RESTATED_DATE])
#Load the balance sheet
balance_sheet = sf.load(dataset='balance',variant='annual', market='us',index=[TICKER], parse_dates=[REPORT_DATE, PUBLISH_DATE, RESTATED_DATE])
#Load the cashflow statement
cashflow_Statement = sf.load(dataset='cashflow',variant='annual', market='us',index=[TICKER], parse_dates=[ PUBLISH_DATE, RESTATED_DATE])
#Load share prices
share_price = sf.load(dataset='shareprices',variant='daily', market='us',index=[TICKER], parse_dates=[DATE])
#Load company details
company_details = sf.load(dataset='companies',variant='annual', market='us', index=[TICKER])

Dataset "us-income-annual" on disk (2 days old).
- Loading from disk ... Done!
Dataset "us-balance-annual" on disk (2 days old).
- Loading from disk ... Done!
Dataset "us-cashflow-annual" on disk (2 days old).
- Loading from disk ... Done!
Dataset "us-shareprices-daily" on disk (2 days old).
- Loading from disk ... Done!
Dataset "us-companies-annual" on disk (2 days old).
- Loading from disk ... Done!


In [4]:
#Get the list from an un-indexed copy of the income statement
ticker_list = list(sf.load(dataset='income',variant='annual', market='us').iloc[:,0])
dis_tckr_list = sorted(list(set(ticker_list))) #List of stocks without repition
print('\n There are', len(dis_tckr_list), 'companies within the database')

Dataset "us-income-annual" on disk (2 days old).
- Loading from disk ... Done!

 There are 2108 companies within the database


In [5]:
# Collect indicators for each stock
def coll_indicators(tckr3):
    year_only = str((date.today() - datetime.timedelta(days=2*365.25)).year -2)

    bal_arr = balance_sheet.loc[tckr3]['Report Date']
    cf_arr = cashflow_Statement.loc[tckr3]['Report Date']
    inc_arr = income_statement.loc[tckr3]['Report Date']

    bal_year, cf_year,inc_year  = [],[],[]

    for i in bal_arr: bal_year.append(str(i).split('-')[0])
    for i in cf_arr: cf_year.append(str(i).split('-')[0])
    for i in inc_arr: inc_year.append(str(i).split('-')[0])

    bal_year = bal_year.index(year_only)
    cf_year = cf_year.index(year_only)
    inc_year = inc_year.index(year_only)

    year1 = str(date.today() - datetime.timedelta(days=2*365.25)) #Data from 2 years ago as last year growth has been impacted by corona virus
    year2 = str(date.today() - datetime.timedelta(days=3*365.25)) #Data from 3 years ago required

    date_arr = share_price.loc[tckr3]['Date']

    y2_index = int(np.where(date_arr == year2)[0])
    y1_index = -1#int(np.where(date_arr == year1)[0])

    #All data collected is from 2018, to predict the 2019 price values
    price = share_price.loc[tckr3]['Close'][y1_index]   #History price
    book_value = balance_sheet.loc[tckr3]['Total Equity'][bal_year]/share_price.loc[tckr3]['Shares Outstanding'][y2_index]  #Book value
    market_cap = share_price.loc[tckr3]['Shares Outstanding'][y1_index]*price   #Market capitalisation
    div_per_share = -cashflow_Statement.loc[tckr3]['Dividends Paid'][cf_year]/share_price.loc[tckr3]['Shares Outstanding'][y1_index]   #Dividend per share
    div_yield = div_per_share/price   #Dividend yield
    eps = income_statement.loc[tckr3][NET_INCOME_COMMON][inc_year]/balance_sheet.loc[tckr3]['Shares (Diluted)'][bal_year] #EPS
    pe_ratio =  price/eps    #P/E ratio
    price_to_book = price/book_value   #Price to book ratio
    curr_ratio = balance_sheet.loc[tckr3]['Total Current Assets'][bal_year]/balance_sheet.loc[tckr3]['Total Current Liabilities'][bal_year] #Current ratio
    quick_ratio = (balance_sheet.loc[tckr3]['Total Current Assets'][bal_year] - balance_sheet.loc[tckr3]['Inventories'][bal_year])/balance_sheet.loc[tckr3]['Total Current Liabilities'][bal_year]          #quick ratio 
    debt_equity = (balance_sheet.loc[tckr3]['Long Term Debt'][bal_year] + balance_sheet.loc[tckr3]['Total Current Assets'][bal_year])/balance_sheet.loc[tckr3]['Total Equity'][bal_year]    #Total debt to equity

    indicators = [tckr3, float(book_value), float(market_cap) , float(div_yield), float(eps), float(pe_ratio), float(price_to_book), float(div_per_share), float(curr_ratio), float(quick_ratio), float(debt_equity), float(price)]

    #Clean data 
    indicators[1:] = [9999 if math.isnan(x) or x == -0.0 or math.isinf(-x) or math.isinf(x) else x for x in indicators[1:]]

    # Comparison of last share price to current, set good and bad (1 and 0) to create target column
    lstyear_price = share_price.loc[tckr3]['Close'][y2_index]
    current_price = share_price.loc[tckr3]['Close'][y1_index]
    price_growth = 100*(float(current_price) - float(lstyear_price))/float(lstyear_price)
    if price_growth > 10:
        rating = 1
    else:
        rating = 0
    indicators.append(rating)

    indicators = np.array(indicators)
    return indicators

In [6]:
# Loop through all companies and create an array of indicators
data = []
counter = 0
for i in dis_tckr_list:
    counter += 1
    try:
        tckr3 = i
        column = coll_indicators(tckr3)
        data.append(column)
    except: pass
    # print(counter,"/", len(dis_tckr_list))
data_clean = pd.DataFrame(data)

In [8]:
print(len(data_clean))

1755


In [ ]:
# Export processed data as csv
compression_opts = dict(method='zip',
                        archive_name='out2.csv')
data_clean.to_csv('out2.zip', index=False,
          compression=compression_opts)